In [18]:
pip install boto3

In [19]:
pip install psycopg2-binary sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pandas sqlalchemy psycopg2


  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\cheta\\anaconda3\\Lib\\site-packages\\psycopg2\\_psycopg.cp312-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [21]:
import json
import pandas
file_path=r"D:\AWS\orders_ETL.json"


In [22]:
f=open(file_path, "r")
data = json.load(f)

In [23]:
data[0]

{'order_id': 1,
 'order_date': '2024-01-10',
 'total_amount': 200.5,
 'customer': {'customer_id': 101,
  'name': 'John Doe',
  'email': 'johndoe@example.com',
  'address': '123 Main St, Springfield'},
 'products': [{'product_id': 'P01',
   'name': 'Wireless Mouse',
   'category': 'Electronics',
   'price': 25.0,
   'quantity': 2},
  {'product_id': 'P02',
   'name': 'Bluetooth Keyboard',
   'category': 'Electronics',
   'price': 45.0,
   'quantity': 1}]}

In [24]:
import pandas as pd

In [25]:
df=pd.read_json(file_path)
df.head()

,order_id,order_date,total_amount,customer,products
0,1,2024-01-10,200.5,"{'customer_id': 101, 'name': 'John Doe', 'emai...","[{'product_id': 'P01', 'name': 'Wireless Mouse..."
1,2,2024-01-12,150.0,"{'customer_id': 102, 'name': 'Jane Smith', 'em...","[{'product_id': 'P03', 'name': 'Laptop Stand',..."
2,3,2024-01-12,120.0,"{'customer_id': 103, 'name': 'Alice Johnson', ...","[{'product_id': 'P04', 'name': 'Gaming Headset..."
3,4,2024-01-13,300.5,"{'customer_id': 101, 'name': 'John Doe', 'emai...","[{'product_id': 'P01', 'name': 'Wireless Mouse..."
4,5,2024-01-14,180.0,"{'customer_id': 104, 'name': 'Bob Brown', 'ema...","[{'product_id': 'P03', 'name': 'Laptop Stand',..."


In [29]:
import json
import pandas as pd
import psycopg2
from urllib.parse import quote
from sqlalchemy import create_engine

# Step 1: Your flattened order data
order_data = []

for order in data:  # 'data' must be your loaded JSON object
    for product in order["products"]:
     order_data.append({
        "order_id": order["order_id"],
        "customer_id": order["customer"]["customer_id"],
        "order_date": order["order_date"],
        "total_amount": order["total_amount"],
        "product_id": product["product_id"],
        "quantity": product["quantity"]
       
    })

# Step 2: Convert to DataFrame
df = pd.DataFrame(order_data)


In [30]:
df.head()


,order_id,customer_id,order_date,total_amount,product_id,quantity
0,1,101,2024-01-10,200.5,P01,2
1,1,101,2024-01-10,200.5,P02,1
2,2,102,2024-01-12,150.0,P03,1
3,3,103,2024-01-12,120.0,P04,2
4,4,101,2024-01-13,300.5,P01,3


In [32]:
# Step 3: PostgreSQL connection details
db_user = "postgres"
raw_password = "Ajjp@121"
db_pass = quote(raw_password)  # Encodes '@' as %40
db_host = "localhost"
db_port = "5432"
db_name = "order"

# Step 4: Create engine using correct credentials
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Step 5: Insert data into existing table named 'orders'
df.to_sql('orders', engine, if_exists='append', index=False)

print("✅ Data inserted successfully into PostgreSQL.")


✅ Data inserted successfully into PostgreSQL.


In [ ]:
df.columns

Index(['order_id', 'customer_id', 'order_date', 'total_amount'], dtype='object')

In [ ]:

product_data=[]
for order in data:  # 'data' must be your loaded JSON object
    for product in order["products"]:
     product_data.append({
        "order_id": order["order_id"],
        "product_id": product["product_id"],
        "product_name": product["name"],
        "category": product["category"],
        "price": product["price"]   

       
    })

# Step 2: Convert to DataFrame
df_product = pd.DataFrame(product_data)
df_product.sort_values(['order_id', 'product_id'], inplace=True)  
df_product.drop_duplicates("product_id", keep='last', inplace=True)  # Keep the first occurrence of each product_id
df_product.drop(columns=['order_id'], inplace=True)  # Drop 'order_id' column

df_product.head()

,product_id,product_name,category,price
18,P03,Laptop Stand,Electronics,75.0
20,P02,Bluetooth Keyboard,Electronics,45.0
19,P04,Gaming Headset,Electronics,60.0
21,P01,Wireless Mouse,Electronics,25.0
22,P05,Desk Organizer,Furniture,30.0


In [38]:
df_product.to_sql('order_products', engine, if_exists='append', index=False)

5

In [44]:
custmor_data=[]
for order in data:  # 'data' must be your loaded JSON object
   
     custmor_data.append({
        "order_id": order["order_id"],
        "customer_id": order["customer"]["customer_id"],
        "customer_name": order["customer"]["name"], 
        "customer_email": order["customer"]["email"],
       
        "customer_address": order["customer"]["address"]
    })
     df_customer= pd.DataFrame(custmor_data)
df_customer.sort_values(['order_id', 'customer_id'], inplace=True)
df_customer.drop_duplicates("customer_id", keep='last', inplace=True)  # Keep the first occurrence of each customer_id
df_customer.drop(columns=['order_id'], inplace=True)  # Drop 'order_id' column
df_customer.head()

,customer_id,customer_name,customer_email,customer_address
1,102,Jane Smith,janesmith@example.com,"456 Oak St, Springfield"
4,104,Bob Brown,bobbrown@example.com,"101 Maple St, Springfield"
5,105,Sania,sania@example.com,"102 Pine St, Springfield"
6,106,Rohit,rohit@example.com,"103 Cedar St, Springfield"
7,107,Nina Patel,ninapatel@example.com,"104 Birch St, Springfield"


In [45]:
 df_customer.head()

,customer_id,customer_name,customer_email,customer_address
1,102,Jane Smith,janesmith@example.com,"456 Oak St, Springfield"
4,104,Bob Brown,bobbrown@example.com,"101 Maple St, Springfield"
5,105,Sania,sania@example.com,"102 Pine St, Springfield"
6,106,Rohit,rohit@example.com,"103 Cedar St, Springfield"
7,107,Nina Patel,ninapatel@example.com,"104 Birch St, Springfield"


In [46]:
df_customer.to_sql('order_customer', engine, if_exists='append', index=False)

9